In [1]:
import numpy as np
import heapq

In [2]:
size = 6
# parking_lot = [['-'] * size] * size # pointers to same objects
parking_lot = []
# creates distinct memory objects at each cell    
parking_lot = [['-' for _ in range(size)] for _ in range(size)]
parking_lot

[['-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-']]

In [3]:
parking_lot[2][0] = parking_lot[2][1] = 'R'
parking_lot[0][1] = parking_lot[1][1] = 'S'
parking_lot[0][2] = parking_lot[0][3] = 'F'
parking_lot[0][4] = parking_lot[0][5] = 'G'
parking_lot[1][4] = parking_lot[1][5] = 'H'
parking_lot[1][3] = parking_lot[2][3] = 'I'
parking_lot[3][0] = parking_lot[4][0] = parking_lot[5][0] = 'C'
# parking_lot[3][0] = parking_lot[4][0] = 'C'
parking_lot[3][1] = parking_lot[3][2] = parking_lot[3][3] = 'B'
# parking_lot[3][1] = parking_lot[3][2] = 'B'
parking_lot[4][2] = parking_lot[5][2] = 'K'
parking_lot[2][4] = parking_lot[3][4] = parking_lot[4][4] = 'A'
# parking_lot[2][4] = parking_lot[3][4] = 'A'
parking_lot[2][5] = parking_lot[3][5] = 'M'
parking_lot[4][5] = parking_lot[5][5] = 'P'
START = [[2, 0], [2, 1]]
EXIT = [3, 5]

In [4]:
parking_lot

[['-', 'S', 'F', 'F', 'G', 'G'],
 ['-', 'S', '-', 'I', 'H', 'H'],
 ['R', 'R', '-', 'I', 'A', 'M'],
 ['C', 'B', 'B', 'B', 'A', 'M'],
 ['C', '-', 'K', '-', 'A', 'P'],
 ['C', '-', 'K', '-', '-', 'P']]

In [5]:
graph = [['-' for _ in range(size)] for _ in range(size)]
graph[2][0] = graph[2][1] = 'R'
graph

[['-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-'],
 ['R', 'R', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-'],
 ['-', '-', '-', '-', '-', '-']]

In [6]:
class node:
    # give front cell row and col
    # give back cell row and col
    # however, after a few move anything can happen so check the actual values when moving
    def __init__(f_r, f_c, b_r, b_c):
        self.f_r = f_r
        self.f_c = f_c
        self.b_r = b_r
        self.b_c = b_c
        self.cost_frm_src = 0
        
    def update_cost_frm_src(self, cost):
        self.cost_frm_src += cost
    
    # move forward one cell in the same row
    def move_forward(self):
        if (self.f_c > self.b_c and self.f_c + 1 < size) or (self.b_c > self.f_c and self.b_c + 1 < size):
            self.b_c += 1
            self.f_c += 1
            
    # move back one cell in the same row
    def move_backward(self):
        if (self.b_c < self.f_c and self.b_c - 1 > -1) and (self.f_c < self.b_c and self.f_c - 1 > -1):
            self.f_c -= 1
            self.b_c -= 1
            
    # shift one row upward
    def move_up(self):
        if self.b_r == self.f_r: # horizontally aligned
            if self.b_r - 1 > -1:
                self.b_r = self.f_r = self.f_r - 1
        else: # vertically aligned
            if (self.b_r < self.f_r and self.b_r - 1 > -1) or (self.f_r < self.b_r and self.f_r - 1 > -1):
                self.b_r -= 1
                self.f_r -= 1
                
    # shift one row downward
    def move_down(self):
        if self.b_r == self.f_r: # horizontally aligned
            if self.b_r + 1 < size:
                self.b_r = self.f_r = self.f_r + 1
        else: # vertically aligned
            if (self.b_r > self.f_r and self.b_r + 1 < size) or (self.f_r > self.b_r and self.f_r + 1 < size):
                self.b_r += 1
                self.f_r += 1
                
#     # pivot on the front cell and rotate 90 upward
#     # alignment will change from horizontal to vertical or from vertical to horizontal
#     # the cell in front will remain in position while the cell behind/below rotates 90 degrees
#     def pivot_back_up(self, counter=True):
#         if self.b_r == self.f_r: #horizontal alignment
#              if self.f_c > self.b_C and self.b_r - 1 > -1:
#                     self.b_c = self.f_c
#                     self.b_r -= 1
#         else: # vertical alignment
#             # it can move either clockwise or counter-clockwise
#             if self.b_r > self.f_r: # b_r is below
#                 if counter and self.b_c + 1 < size:
#                     self.b_r = self.f_r
#                     self.b_c += 1
#                 elif not counter and self.b_c - 1 > -1:
#                     self.b_r = self.f_r
#                     self.b_c -= 1
#             else:
#                 if counter and self.f_c + 1 < size:
#                     self.f_r = self.b_r
#                     self.f_c += 1
#                 elif not counter and self.f_c - 1 > -1:
#                     self.f_r = self.b_r
#                     self.f_c -= 1

    def vertical_2_horizontal(self, forward=True):
        # the back cell can rotate upward (counter- and clockwise)
        # the front cell can rotate downward (counter- and clockwise)
        if self.b_r > self.f_r: # b_r is below
            if forward and self.f_c + 1 < size: # prefer to move forward
                self.b_r = self.f_r
                self.b_c += 1
            elif not forward and self.f_c - 1 > -1:
                self.b_r = self.f_r
                self.b_c += 1
        else:
            if forward and self.b_c + 1 < size: # prefer to move forward
                self.f_r = self.b_r
                self.f_c += 1
            elif not forward and self.b_c - 1 > -1:
                self.f_r = self.b_r
                self.f_c += 1
    
    def horizontal_2_vertical(self, up=True):
        if self.b_c > self.f_c: # b_c is in front
            if up and self.b_r - 1 > -1: # arbitrarily chosing to move upward preferentially
                self.b_c = self.f_c
                self.b_r -= 1
            elif not up and self.b_c + 1 < size:
                self.b_c = self.f_c
                self.b_r += 1
        else:
            if up and self.f_r - 1 > -1: # arbitrarily chosing to move upward preferentially
                self.f_c = self.b_c
                self.f_r -= 1
            elif not up and  self.f_c + 1 < size:
                self.f_c = self.b_c
                self.f_r += 1

In [7]:
def cost(node, dst, src):
    # Manhattan distance
    return np.abs(node[0] - dst[0]) + np.abs(node[1] - dst[1]) + np.abs(node[0] - src[0]) + np.abs(node[1] - src[1])

In [8]:
SRC = [2, 1]
DST = [3, 5]
node = [2, 2]
cost(node, DST, SRC)

5

In [9]:
SRC = [2, 1]
DST = [3, 5]
COSTS = [[np.inf for _ in range(size)] for _ in range(size)]
COSTS[SRC[0]][SRC[1]] = 0
i = SRC[0]
j = SRC[1]
for i in range(size):
    for j in range(size):
        COSTS[i][j] = cost([i, j], SRC, DST)
COSTS[DST[0]][DST[1]] = 0

In [10]:
COSTS

[[11, 9, 9, 9, 9, 9],
 [9, 7, 7, 7, 7, 7],
 [7, 5, 5, 5, 5, 5],
 [7, 5, 5, 5, 5, 0],
 [9, 7, 7, 7, 7, 7],
 [11, 9, 9, 9, 9, 9]]

In [11]:
def get_neighborhood(pos, size):
    """pos should be a list of 2 lists indicating position of the red car
    example: [[2, 1], [3, 1]]
    size is simply the size of grid
    # visited contains elements we don't want to add in the neighborhood, typically because we have already seen them
    """
#     assert isinstance(pos, list)
#     assert len(pos) == 2
#     assert isinstance(pos[0], list)
#     assert len(pos[0]) == 2
#     assert isinstance(pos[1], list)
#     assert len(pos[1]) == 2
    n = []
    f_r, f_c = pos[0]
    b_r, b_c = pos[1]
    
    # Assumption is car can't move diagonally and hence will never be diagonally aligned
    
    if f_r == b_r: # same row => car is horizontally aligned
        if f_c < b_c: # front cell is actually back cell
            # swap cells
            t_r = f_r
            t_c = f_c
            f_r = b_r
            f_c = b_c
            b_r = t_r
            b_c = t_c
        if f_r + 1 < size and b_r + 1 < size: # row below is valid
            n.append([[f_r + 1, f_c], [b_r + 1, b_c]]) # shift down a row
            n.append([[f_r, f_c], [b_r + 1, f_c]]) # rotate down 90 degrees pivoting on the front cell
            n.append([[f_r + 1, b_c], [b_r, b_c]]) # rotate down 90 degrees pivoting on the back cell
        if f_r - 1 > -1 and b_r - 1 > -1: # row above is valid
            n.append([[f_r - 1, f_c], [b_r - 1, b_c]]) # shift up a row
            n.append([[f_r, f_c], [b_r - 1, f_c]]) # rotate up 90 degrees pivoting on the front cell
            n.append([[f_r - 1, b_c], [b_r, b_c]]) # rotate up 90 degrees pivoting on the back cell
        if f_c + 1 < size and b_c + 1 < size: # next column is valid
            n.append([[f_r, f_c + 1], [b_r, b_c + 1]]) # move ahead by one column
        if b_c - 1 > -1 and f_c - 1 > -1: # previous column is valid
            n.append([[f_r, f_c - 1], [b_r, b_c - 1]]) # move back by one column
    else: # car is vertically aligned
        if f_r > b_r: # f is the bottom cell
            t_r, t_c = b_r, b_c # top cell
            b_r, b_c = f_r, f_c # bottom cell
        else:
            t_r, t_c = f_r, f_c # top cell
        if t_c - 1 > -1 and b_c - 1 > -1: # previous column is valid
            n.append([[t_r, t_c - 1], [b_r, b_c - 1]]) # shift to the previous column
            n.append([[b_r, t_c - 1], [b_r, b_c]]) # rotate left by 90 degrees pivoting on the bottom cell
            n.append([[t_r, t_c], [t_r, b_c - 1]]) # rotate left by 90 degrees pivoting on the top cell
        if t_c + 1 < size and b_c + 1 < size: # next column is valid
            n.append([[t_r, t_c + 1], [b_r, b_c + 1]]) # shift to the next column
            n.append([[b_r, t_c + 1], [b_r, b_c]]) # rotate right by 90 degrees pivoting on the bottom cell
            n.append([[t_r, t_c], [t_r, b_c + 1]]) # rotate right by 90 degrees pivoting on the top cell
        if t_r - 1 > -1 and b_r - 1 > -1: # the row above is valid
            n.append([[t_r - 1, t_c], [b_r - 1, b_c]]) # move up by one row
        if b_r + 1 < size and t_r + 1 < size: # the row below is valid
            n.append([[t_r + 1, t_c], [b_r + 1, b_c]]) # move down by one row
            
#     neighborhood = [elem for elem in n if elem not in visited]
    return n

In [12]:
def astar(src, dst, costs, size):
    """
    @inputs:
        src: the starting point
        dst: the destination point
        costs: costs of each cell (as measured by the cost function)
        size: the size of the grid
    @output:
        list containing cells that constitute the least cost path
    """
    frontier = []
    heapq.heapify(frontier)
    c = 0.5 * (costs[src[0][0]][src[0][1]] + costs[src[0][0]][src[0][1]])
    heapq.heappush(frontier, (c, src))
    path = []
    visited = []
    while frontier:
        pos = heapq.heappop(frontier)[1]
        path.append(pos)
        if pos[0] == dst or pos[1] == dst:
            path.append(pos)
            break
        neighborhood = get_neighborhood(pos, size)
        for n in neighborhood:
            if n not in visited:
                c = 0.5 * (costs[n[0][0]][n[0][1]] + costs[n[0][0]][n[0][1]])
                heapq.heappush(frontier, (c, n))
                visited.append(n)
    return path

In [13]:
SRC = [[2, 0], [2, 1]]
astar(SRC, DST, COSTS, size)

[[[2, 0], [2, 1]],
 [[2, 1], [1, 1]],
 [[2, 1], [3, 1]],
 [[2, 1], [2, 0]],
 [[2, 1], [2, 2]],
 [[2, 2], [1, 2]],
 [[2, 2], [2, 1]],
 [[2, 2], [3, 2]],
 [[2, 2], [2, 3]],
 [[2, 3], [1, 3]],
 [[2, 3], [2, 2]],
 [[2, 3], [3, 3]],
 [[2, 3], [2, 4]],
 [[2, 4], [1, 4]],
 [[2, 4], [2, 3]],
 [[2, 4], [3, 4]],
 [[3, 5], [3, 4]],
 [[3, 5], [3, 4]]]